<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_DMM/blob/main/Conv1DLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import tensorflow as tf
import pandas as pd
import numpy as np

tf.random.set_seed(1)

#Data Load
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/test_v.txt')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/train_v.txt')
train_data_input = df_train.iloc[:,:-1]
train_data_output = df_train.iloc[:,-1]
test_data_input = df_test.iloc[:,:-1]
test_data_output = df_test.iloc[:,-1]

train_data = df_train.iloc[:,:]
test_data = df_test.iloc[:,:]

trainSet_for_result = train_data
testSet_for_result = test_data

print(train_data_input.shape,train_data_input)
print(train_data_output.shape, train_data_output)
print("train_data : ",train_data.shape,train_data)
print("test_data : ",test_data.shape,test_data)

In [ ]:
#Normalization
def MinMaxScaler(data):
  denom = np.max(data,0) - np.min(data,0)
  nume = data-np.min(data,0)
  return nume/denom

def back_MinMax(data,max,min):
  diff = max-min
  back = data * diff + min
  return back

dataDim = train_data_input.shape[1]

print(dataDim)
test_data_input_max = np.max(test_data_input)
train_data_input_max = np.max(train_data_input)
test_data_output_max = np.max(test_data_output)
train_data_output_max = np.max(train_data_output)

test_data_input_min = np.min(test_data_input)
train_data_input_min = np.min(train_data_input)
test_data_output_min = np.min(test_data_output)
train_data_output_min = np.min(train_data_output)

print(test_data_output_max)
print(test_data_output_min)

train_data_input = MinMaxScaler(train_data_input)
train_data_output = MinMaxScaler(train_data_output)
test_data_input = MinMaxScaler(test_data_input)
test_data_output = MinMaxScaler(test_data_output)

train_data = MinMaxScaler(train_data)
test_data = MinMaxScaler(test_data)



In [ ]:
# Data Dimension matching for CNN
print(train_data_input.shape,type(train_data_input))
train_data_input_np = train_data_input.to_numpy()
print(train_data_input_np.shape,type(train_data_input_np))
train_data_input_np2 = train_data_input_np.reshape((train_data_input_np.shape[0],train_data_input_np.shape[1],1))
train_data_output_np2 = train_data_output.to_numpy()
print(train_data_input_np2.shape)
print(train_data_output_np2.shape)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Hyper Parameter
filters = 128
kernel_size = 3
pooling_size = 3
epochs = 200
stride = 1
input_shape = train_data.shape[1]
outputDim = input_shape
hiddenDim = 100
initializer = tf.keras.initializers.GlorotNormal()

#LSTM after CNN
model = keras.Sequential()
model.add(Conv1D(filters = filters, kernel_size = kernel_size, strides = stride, activation = 'relu', input_shape = (input_shape,1)))
model.add(MaxPooling1D(pool_size = pooling_size))
model.add(Flatten())
model.add(layers.Reshape((filters, 1)))
model.add(layers.LSTM(units = hiddenDim, activation='tanh',input_shape = (None,filters, 1), return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim, activation = 'tanh',return_sequences = False, kernel_initializer = initializer))
model.add(layers.Dense(outputDim, activation='linear'))
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
#Training
hist = model.fit(train_data,train_data, epochs = epochs, verbose = 0)

In [ ]:
#Evaluating
batch_size = 16

res = model.evaluate(test_data, test_data, batch_size = batch_size)
print("Accuracy : ",res)

estimated = model.predict(test_data, verbose = 0)
estimated = back_MinMax(estimated, test_data_output_max, test_data_output_min)
estimated = estimated[:,-1]
real = testSet_for_result.iloc[:,-1] # estimated + seqLength

print("Estimated : ",estimated.shape, estimated)
print("Real : ",real.shape, real)

In [ ]:
#Visualization
import matplotlib.pyplot as plt
plt.figure()
plt.plot(estimated[:len(estimated)],label = "Estimated by Conv1D,LSTM Mixture Model")
plt.plot(real[:len(real)], label = "Real")
plt.legend(prop = {'size':10})

print("Estimated : ",estimated.shape,estimated)
print("Real : ",real.shape, real)